In [1]:
#ENV["JULIA_NUM_THREADS"] = "4" # Itt állítsd be a kívánt szálak számát
using Base.Threads
@info "A szkript futási környezete" nthreads()

┌ Info: A szkript futási környezete
└   nthreads() = 4


In [2]:
using LowLevelFEM, LinearAlgebra
gmsh.initialize()
gmsh.open("beam.geo")

Info    : Reading 'beam.geo'...
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 60%] Meshing curve 8 (Line)
Info    : [ 70%] Meshing curve 9 (Line)
Info    : [ 80%] Meshing curve 10 (Line)
Info    : [ 90%] Meshing curve 11 (Line)
Info    : [100%] Meshing curve 12 (Line)
Info    : Done meshing 1D (Wall 0.00386734s, CPU 0.001289s)
Info    : Meshing 2D...
Info    : [  0%] Meshing surface 1 (Transfinite)
Info    : [ 20%] Meshing surface 2 (Transfinite)
Info    : [ 40%] Meshing surface 3 (Transfinite)
Info    : [ 60%] Meshing surface 4 (Transfinite)
Info    : [ 70%] Meshing surface 5 (Transfinite)
Info    : [ 90%] Meshing surface 6 (Transfinite)
Info    : Done meshing 2D (Wall 0.000620742s, CPU 0.000618s)
Info    : Meshing 3D..

In [3]:
mat = material("body", E=2e5, ν=0.3)
cantilever = Problem([mat], type=:Solid)

Problem("beam", :Solid, 3, 3, LowLevelFEM.Material[LowLevelFEM.Material("body", :Hooke, 200000.0, 0.3, 7.85e-9, 45.0, 4.2e8, 1.2e-5, 115384.61538461536, 76923.07692307692, 166666.66666666666, 1.0e-7, 0.1, 1.0)], 1.0, 6561, LowLevelFEM.Geometry("", "", 0, 0, nothing, nothing, nothing, nothing))

In [4]:
force = load("top", fy=-1)
supp = displacementConstraint("left", ux=0, uy=0, uz=0)

("left", 0, 0, 0)

In [5]:
K = stiffnessMatrix(cantilever)

19683×19683 SparseArrays.SparseMatrixCSC{Float64, Int64} with 3143411 stored entries:
⎡⡿⣯⣿⠿⠿⢿⣬⣤⣄⣿⠛⣥⣽⡿⢈⣹⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣶⣾⣿⎤
⎢⣿⡟⢿⣷⡲⠾⠙⠉⠉⣿⡮⠁⢹⡿⠘⠻⢿⡻⠽⢿⣿⣟⣛⣛⣛⡛⠛⠛⠛⠛⠛⠻⠿⠭⠭⠭⠭⠭⠭⠿⎥
⎢⣿⣇⣸⡎⠻⣦⣀⡀⠀⣇⢧⡀⣸⣹⢀⣀⣈⠻⣄⣀⣀⣀⣀⡈⠉⠉⠉⠉⠛⠓⠒⠒⠶⠦⠤⠤⣤⣄⣀⣀⎥
⎢⠂⣿⡗⠀⠀⠸⡿⣯⡙⢺⠀⢿⡗⡇⢸⣿⠺⣍⠓⠒⠒⠾⠿⠭⠭⢭⣭⣉⣉⣉⣉⠉⠉⠉⠉⠉⠙⠛⠓⠒⎥
⎢⣤⣽⣧⣤⠤⢤⣳⣈⣻⣾⣤⣼⣷⠇⠀⢿⣤⣼⣷⣤⣤⣤⣤⣤⣤⣤⣤⣤⣤⣬⣭⣭⣭⣽⣿⣶⣶⣶⣶⣤⎥
⎢⠟⣤⠎⠋⠉⠳⣤⣄⣀⣿⢿⣷⡄⠀⠀⠰⢾⣻⡷⠶⠾⠿⢿⣿⣛⣛⣛⣛⣛⣿⣭⣭⣭⣭⣭⣽⣷⣶⣶⠶⎥
⎢⣷⡿⣷⡶⣖⣺⠽⠭⠽⠟⠀⠉⢻⣶⣶⢾⣿⣿⣿⣿⣿⠿⠿⠿⠿⠿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⎥
⎢⣆⣰⣶⡀⠀⢰⣶⣶⣤⣄⢀⡀⣸⣟⢿⣷⣾⣿⣷⡾⠿⠿⠿⢿⣟⣛⣛⣛⣿⣯⣭⣭⣭⣭⣿⣿⣶⣶⣶⣶⎥
⎢⣿⣿⣿⡳⣦⡘⡞⢦⣀⣿⣾⣳⣿⣿⣾⣿⠻⣦⡘⠛⠒⠒⠒⠶⠶⠤⠤⠤⣤⣤⣀⣀⣀⣀⣀⠀⠀⠀⠈⠉⎥
⎢⣿⣿⣷⣇⠀⢹⢹⠀⠙⣿⢹⡏⣿⣿⣹⡿⣶⠈⢻⣶⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⠉⠉⠉⠛⠓⎥
⎢⣿⣿⣿⢿⠀⢸⣸⡄⠀⣿⣾⡇⣿⡟⣿⡇⢸⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⢸⡀⠸⡟⡇⠀⣿⣿⣷⣿⡇⣿⣇⢸⡄⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⠸⡇⠀⡇⣇⠀⣿⣿⢸⣿⡇⣿⢹⠘⡇⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⠀⡇⠀⡇⢻⠀⣿⣿⢸⣿⣿⣿⢸⠀⡇⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⠀⢿⠀⡇⢸⡀⣿⣿⣼⣿⣿⡿⣿⠀⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⣿⡀⢸⠀⡇⠘⡇⣿⡇⣿⣿⣿⡇⣿⠀⢸⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⣿⣿⡟⡇⠸⡇⡇⠀⣇⣿⡇⣿⣿⣿⡇⣿⠀⢸⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⠀⠀⎥
⎢⣿⣿⡇⡇⠀⡇⡇⠀⢻⣿⣇⣿⣿⣿⣿⣿⠀⠘⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⠀⠀⎥
⎢⢻⣿⡇⡇⠀⢿⣷⠀⢸⣿⢹⣿⣿⣿⢸⣿⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⣄⠀⎥
⎣⣾⣿⣧⡇⠀⢸⢹⠀⠘⣿⢸⡟⣿⣿⢸⣿⡆⠀⢿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣷⎦

In [6]:
using BenchmarkTools

In [7]:
#@time K = LowLevelFEM.stiffnessMatrixSolid(cantilever)

In [8]:
#@time K = LowLevelFEM.stiffnessMatrixSolidParallel(cantilever)

In [9]:
f = loadVector(cantilever, [force])

VectorField

19683×1 Matrix{Float64}:
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -0.17361111111110483
  0.0
  0.0
 -0.17361111111111074
  0.0
  0.0
  ⋮
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0

In [10]:
applyBoundaryConditions!(K, f, [supp])

In [11]:
E = mat.E
ν = mat.ν

0.3

In [12]:
u = K \ f
A = (u ∘ ∇ + ∇ ∘ u) / 2
I = unitTensor(A)
S = E / (1 + ν) * (A + ν / (1 - 2ν) * trace(A) * I);

In [13]:
showDoFResults(u, :vector, visible=true)
Sz = showDoFResults(S, :sz)
Syz = showDoFResults(S, :syz)

3

In [14]:
plotOnPath("path", Sz)
plotOnPath("path", Syz)

5

In [15]:
b = -S ⋅ ∇

VectorField

640-element Vector{Matrix{Float64}}:
 [10.500824578777378; -2.4085311217806717; … ; 7.079561678632796; 5.3664627481392815;;]
 [-0.9288264754115029; -11.228771025827037; … ; 7.15529913345551; 1.4457669209214978;;]
 [0.04822571501901862; -13.338663363467845; … ; 7.155299133442719; 1.4457669209138482;;]
 [13.166643310847103; -13.300203363157724; … ; 7.079561678596548; 5.366462748102203;;]
 [-0.5030595143026081; 12.366622019724055; … ; -0.6498141117196647; 0.4262709244574956;;]
 [-0.2819119183415623; 2.3556635789258222; … ; -1.1757340208124205; -1.6718932853116686;;]
 [-0.0008463017264759357; -0.1685868279643623; … ; -1.175734020808803; -1.6718932852996415;;]
 [0.29354070556106837; -0.21558499014019183; … ; -0.6498141117040441; 0.42627092445285997;;]
 [-9.863590763001877; 12.858993214140405; … ; -0.6683919834968579; -0.396947326140098;;]
 [0.07354386794481128; 3.496633821212735; … ; -1.192506869336528; 1.6864594348988604;;]
 [-0.05080694485213755; 1.2230114036222903; … ; -1.192506869336776

In [16]:
showDoFResults(b, :vector)

6

In [17]:
openPostProcessor()

-------------------------------------------------------
Version       : 4.15.0-git
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib tinyobjloader
FLTK version  : 1.3.8
OCC version   : 7.9.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
Fontconfig warning: using without calling FcInit()


In [18]:
gmsh.finalize()